In [5]:
from gurobipy import *
import sympy as sp
from sympy.abc import x, y, z, u
import csv
from tqdm import tqdm
variables = [x, y, z, u]

def compute_weight(f, weight):
    f = sp.Poly(f, gens = variables)
    return min([sum([weight[i]*degrees[i] for i in range(len(degrees))]) for degrees in f.monoms()])

def find_wt(constraints, solution):
    try:
    # Create a new model
        model = Model("mip1")
        
        #search all solution
        model.Params.PoolSearchMode = 2
        model.Params.PoolSolutions = 10000
        model.Params.LogToConsole = 0
        
        # Create variables
        f = model.addVar(lb=1, ub=GRB.INFINITY, vtype=GRB.INTEGER, name="f")
        x = model.addVar(lb=1, ub=GRB.INFINITY, vtype=GRB.INTEGER, name="x")
        y = model.addVar(lb=1, ub=GRB.INFINITY, vtype=GRB.INTEGER, name="y")
        z = model.addVar(lb=1, ub=GRB.INFINITY, vtype=GRB.INTEGER, name="z")
        u = model.addVar(lb=1, ub=GRB.INFINITY, vtype=GRB.INTEGER, name="u")
            
        # Set objective
        model.setObjective(f , GRB.MAXIMIZE)

        #discrepancy
        alpha = 1
        
        
        # equal constraint
        model.addConstr(x + y + z + u - f - 1, GRB.EQUAL, alpha , "c0")
        
        a = [x, y, z, u]
        # constraints
        for degree in constraints:
            degree = list(degree)
            h = 0
  
            for i in range(len(degree)):
                h+=a[i]*degree[i]

            model.addConstr(h , GRB.GREATER_EQUAL, f,str(degree))

        #RUN Algorithm
        model.optimize()
        
        nSolutions = model . SolCount
            
        for e in range ( nSolutions ):
            model . setParam (GRB. Param . SolutionNumber , e)
            sol = tuple([int(round(x)) for x in model.Xn])
            
            if sol not in solution:
                solution.append(sol)

    except GurobiError:
        print('Error reported')

In [6]:
#define our first example
f = 'x**2+y**3+z**4+z**2*u**2+u**8'

#translate it into the polynomial in sympy
f = sp.Poly(f)

In [7]:
solution = []
constraints = f.monoms()

# find out all possible weights
find_wt(constraints, solution)
solution = list(set(solution))

Academic license - for non-commercial use only
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolSolutions to 10000
   Prev: 10  Min: 1  Max: 2000000000  Default: 10


In [8]:
solution = [weight for weight in solution if weight[0] == compute_weight(f, list(weight)[1:])]


solution.sort(key = lambda s: s[0], reverse = True)
print('the number of admissible weights is',len(solution))
print('all possibilities are ',solution)

admissible_weights = [list(weight)[1:] for weight in solution]
print('all admissible weights are', admissible_weights)

the number of admissible weights is 4
all possibilities are  [(6, 3, 2, 2, 1), (4, 2, 2, 1, 1), (3, 2, 1, 1, 1), (2, 1, 1, 1, 1)]
all admissible weights are [[3, 2, 2, 1], [2, 2, 1, 1], [2, 1, 1, 1], [1, 1, 1, 1]]
